<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2021notebooks/2021_0613gen_sentences_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import time
np.random.seed(1)   #乱数を発生させるための種を定める

# 1. 語彙の定義

In [ ]:
Nsingle = ('boy', 'girl', 'cat', 'dog')      #名詞単数形
Nplural = ('boys', 'girls', 'cats', 'dogs')  #名詞複数形
NpropN = ('Mary', 'John')                    #人称名詞
VIsingle = ('walks', 'lives')                #自動詞 3人称単数動詞
VIplural = ('walk', 'live')                  #自動詞の原形
VTsingle = ('chases', 'feeds')               #他動詞 3人称単数形
VTplural = ('chase', 'feed')                 #他動詞の原形
VAsingle = ('sees', 'hears')                 #他動詞，目的語をとっても取らなくても良い動詞の 3 人称単数形
VAplural = ('see', 'hear')                   #他動詞，目的語をとっても取らなくても良い動詞の原形

# 2. 下請け関数の定義

In [ ]:
def flatten(l):
    """入れ子になったリストの要素取り出して並べる"""
    ret = ""
    if isinstance(l, str):
        ret += l + ' '
    else:
        if isinstance(l, list):
            for _l in l:
                ret += flatten(_l)
    return ret

def gen_NpropN():    return NpropN[np.random.randint(0, len(NpropN))]
def gen_Nsingle():    return Nsingle[np.random.randint(0, len(Nsingle))]
def gen_Nplural():    return Nplural[np.random.randint(0, len(Nplural))]
def gen_VIsingle():    return VIsingle[np.random.randint(0, len(VIsingle))]
def gen_VIplural():    return VIplural[np.random.randint(0, len(VIplural))]
def gen_VTsingle():    return VTsingle[np.random.randint(0, len(VTsingle))]
def gen_VTplural():    return VTplural[np.random.randint(0, len(VTplural))]
def gen_VAsingle():    return VAsingle[np.random.randint(0, len(VAsingle))]
def gen_VAplural():    return VAplural[np.random.randint(0,len(VAplural))]

In [ ]:
def make_a_sentence():
    """文生成関数"""
    line = []
    
    # 名詞節の作成
    Phrase = gen_NP()
    
    # 乱数を発生させて 1, 2, 3 を決める
    # この乱数によって後続する動詞の種類，すなわち文型を場合分けして処理する
    x = np.random.randint(1, 3+1)
    
    if x == 1:                           # 自動詞節の場合
        is_plural = Phrase.pop(0)        # 名詞節の先頭要素を取り出す
        line.append(Phrase)              # 名詞節の末尾に要素を追加する
        if is_plural == 'single':        # 単数形が要求される場合
            line.append(gen_VIsingle())  # 自動詞の単数形追加する
        else:
            line.append(gen_VIplural())
    elif x == 2:                         # 他動詞節の場合
        is_plural = Phrase.pop(0)
        line.append(Phrase)
        if is_plural == 'single':
            line.append(gen_VTsingle())
        else:
            line.append(gen_VTplural())
        _Phrase = gen_NP()
        is_plural = _Phrase.pop(0)
        line.append(_Phrase)
    else:                                # 目的語を取ってもも取らなくても良い場合
        is_plural = Phrase.pop(0)
        line.append(Phrase)
        if is_plural == 'single':        # 自動詞的用法
            line.append(gen_VAsingle())
        else:
            line.append(gen_VAplural())  # 他動詞的用法
        xx = np.random.randint(1, 2+1)
        if xx == 1:
            _Phrase = gen_NP()
            is_plural = _Phrase.pop(0)
            line.append(_Phrase)

    return line

In [ ]:
def gen_NP():
    """名詞句生成"""
    phrase = []

    # 1, 2, 3 のいずれか生成する
    x = np.random.randint(1, 3+1)           
    if x == 1:                               # 名詞の場合
        if np.random.randint(1, 2+1) == 1:   # 名詞が単数形の場合
            phrase.append('single')
            phrase.append(gen_Nsingle())
        else:                                # 複数形の場合
            phrase.append('plural')
            phrase.append(gen_Nplural())
    elif x == 2:                             # 名詞 + 関係代名詞節の場合
        if np.random.randint(1, 2+1) == 1:   # 名詞が単数形の場合
            phrase.append('single')
            phrase.append(gen_Nsingle())
            phrase.append(gen_RC('single'))  # 関係代名詞節の追加
        else:
            phrase.append('plural')          # 名詞が複数形の場合
            phrase.append(gen_Nplural())
            phrase.append(gen_RC('plural'))
    else:                                     # その他
        phrase.append('single')
        phrase.append(gen_NpropN())

    return phrase

def gen_RC(mode):
    """関係代名詞節生成"""
    ret = []

    ret.append("who")
    x = np.random.randint(1,3+1)
    if x == 1:                             # who + 自動詞の場合
        if mode == 'single':
            ret.append(gen_VIsingle())
        else:
            ret.append(gen_VIplural())
    elif x == 2:                           # who + 他動詞 + 名詞節の場合
        if mode == 'single':
            ret.append(gen_VTsingle())
        else:
            ret.append(gen_VTplural())
        tmp = gen_NP()
        is_plural = tmp.pop(0)
        ret.append(tmp)
    else:                                  # who + 他動詞 + (名詞節)の場合
        if mode == 'single':
            ret.append(gen_VAsingle())
        else:
            ret.append(gen_VAplural())
        xx = np.random.randint(1,2+1)
        if xx == 1:
                tmp = gen_NP()
                is_plural = tmp.pop(0)
                ret.append(tmp)
    return ret


# 3. 文生成の実行

In [ ]:
N = 100  # 生成したい文章の数を決める
for _ in range(N):
    print(flatten(make_a_sentence()))